# Importing Libraries


In [11]:
import pandas as pd
import numpy as np


# Data PreProcessing


In [12]:
df=pd.read_csv('diabetes.csv') 

In [13]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
df.shape

(768, 9)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [16]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

# As No Null Value Exists. PreProcessing not Required.
## But some Columns have values 0. Which is not possible.
### Lets Deal with That

In [32]:
print("Zero Values")
for col in df.columns.values:
    sum=0
    for i in range(0,len(df[col])):
        if(0==df[col][i]):
            sum+=1
    print(f"{col} :",sum)

Zero Values
Pregnancies : 111
Glucose : 5
BloodPressure : 35
SkinThickness : 227
Insulin : 374
BMI : 11
DiabetesPedigreeFunction : 0
Age : 0
Outcome : 500


# Here Glucose , BloodPressure , SkinThickness, Insulin and BMI cannot Have Value 0
## Appending them With Mean Value maybe a good

In [33]:
cols=['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for i in range(0,len(cols)):
    df[cols[i]].replace(0,df[cols[i]].median(),inplace=True)

In [34]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,30.5,33.6,0.627,50,1
1,1,85,66,29,30.5,26.6,0.351,31,0
2,8,183,64,23,30.5,23.3,0.672,32,1
3,1,89,66,23,94.0,28.1,0.167,21,0
4,0,137,40,35,168.0,43.1,2.288,33,1


# Now to Feature Selection

In [35]:
df_comp=df.loc[:,df.columns!='Outcome']
df_comp.corrwith(df['Outcome'])

Pregnancies                 0.221898
Glucose                     0.492782
BloodPressure               0.165723
SkinThickness               0.189065
Insulin                     0.148457
BMI                         0.312249
DiabetesPedigreeFunction    0.173844
Age                         0.238356
dtype: float64

# BloodPressure and SkinThickness are Weakly correlated
    So droping them

In [40]:
print(df_comp.corrwith(df['Outcome']).median())

0.2054817863046901


# Dropping Correlation less than 0.205

In [42]:
cols=['BloodPressure','SkinThickness','Insulin','DiabetesPedigreeFunction']
for i in range(0,len(cols)):
    df.drop(cols[i],axis=1,inplace=True)

In [43]:
df.head()

,Pregnancies,Glucose,BMI,Age,Outcome
0,6,148,33.6,50,1
1,1,85,26.6,31,0
2,8,183,23.3,32,1
3,1,89,28.1,21,0
4,0,137,43.1,33,1


In [44]:
df.corr()

,Pregnancies,Glucose,BMI,Age,Outcome
Pregnancies,1.000000,0.128213,0.021546,0.544341,0.221898
Glucose,0.128213,1.000000,0.231400,0.266909,0.492782
BMI,0.021546,0.231400,1.000000,0.025744,0.312249
Age,0.544341,0.266909,0.025744,1.000000,0.238356
Outcome,0.221898,0.492782,0.312249,0.238356,1.000000


# Age and Pregnacies Seem to be strongly correlated with 0.544

Checking corr of each with Outcome

In [45]:
print("Pregnancies")
print(df['Pregnancies'].corr(df['Outcome']))
print("Age")
print(df['Age'].corr(df['Outcome']))

Pregnancies
0.22189815303398652
Age
0.23835598302719768


# As Age is more correlated with Outcome. Dropping Pregnancies

In [46]:
df.drop('Pregnancies',axis=1,inplace=True)

In [47]:
df.corr()

,Glucose,BMI,Age,Outcome
Glucose,1.000000,0.231400,0.266909,0.492782
BMI,0.231400,1.000000,0.025744,0.312249
Age,0.266909,0.025744,1.000000,0.238356
Outcome,0.492782,0.312249,0.238356,1.000000


In [48]:
df.head()


,Glucose,BMI,Age,Outcome
0,148,33.6,50,1
1,85,26.6,31,0
2,183,23.3,32,1
3,89,28.1,21,0
4,137,43.1,33,1


# Now building Model

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import GridSearchCV

# Now  Implementing Machine Learning


In [50]:
svc=SVC()
knn=KNeighborsClassifier()
dtc=DecisionTreeClassifier()
nb=GaussianNB()

In [51]:
models={'svc':svc,'knn':knn,'dtc':dtc,'nb':nb}

In [52]:
x,y=df.drop('Outcome',axis=1),df['Outcome']

In [53]:
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=200)

In [54]:
model_accuracy={}

for model in models.keys():
    mod = models[model].fit(train_x,train_y)
    predict=mod.predict(test_x)
    #print(f"{model}",classification_report(predict,test_y))
    print(f"{model} ",confusion_matrix(predict,test_y))
    
    if model not in model_accuracy.keys():
        model_accuracy.update({model:accuracy_score(test_y,predict)})

svc [[98 21]
 [ 8 27]]
knn [[87 18]
 [19 30]]
dtc [[75 19]
 [31 29]]
nb [[96 19]
 [10 29]]


In [55]:
model_accuracy

{'svc': 0.8116883116883117,
 'knn': 0.7597402597402597,
 'dtc': 0.6753246753246753,
 'nb': 0.8116883116883117}

# Lets try With GridSearch


In [56]:
params={
    'knn': {'n_neighbors':[3,5,7,9,11,13,15],
    'metric':['cosine','euclidean','manhattan'],
    'weights':['uniform','distance']},
    
    'svc': {
    'C':[0.1,1,10,100], 
    'gamma':[1,0.1,0.01,0.01], 
    'kernel':['rbf','linear']},
    
    'dtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    },
    
    'nb':{
        'priors': [None],
    'var_smoothing': [0.00000001, 0.000000001, 0.00000001]
    }
    
    }

In [57]:
model_accuracy={}
for model in models.keys():
    mod = GridSearchCV(
    models[model],
    params[model],
    verbose=1, #Progress bar showing
    cv=20, #cross validation
    n_jobs=-1, #cores to assign
)
    gridsearch_result=mod.fit(train_x,train_y)
    predict=mod.predict(test_x)
    print(f"{model} : ",gridsearch_result.best_estimator_)
    if model not in model_accuracy.keys():
        model_accuracy.update({model:accuracy_score(test_y,predict)})

Fitting 20 folds for each of 32 candidates, totalling 640 fits
svc :  SVC(C=10, gamma=1, kernel='linear')
Fitting 20 folds for each of 42 candidates, totalling 840 fits
knn :  KNeighborsClassifier(metric='euclidean', n_neighbors=15)
Fitting 20 folds for each of 12 candidates, totalling 240 fits
dtc :  DecisionTreeClassifier(max_depth=4)
Fitting 20 folds for each of 3 candidates, totalling 60 fits
nb :  GaussianNB(var_smoothing=1e-08)


In [58]:
model_accuracy

{'svc': 0.8116883116883117,
 'knn': 0.7727272727272727,
 'dtc': 0.7467532467532467,
 'nb': 0.8116883116883117}